In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
import time

import numpy as np
import tensorflow as tf

#from tensorflow.python.framework import ops
#ops.reset_default_graph()
#sess = tf.Session()
import utils

Using TensorFlow backend.


In [2]:
data_folder = "../data/vardial2018-sample"
#data_files = ["EGY", "GLF", "LAV", "MSA", "NOR"]
train_file = "train.words"
dev_file = "dev.words"
batch_size = 10

# Model Hyperparameters
# Embeddings
vocab_size = 50000
embed_size = 128

# Convolutional layers
filter_sizes = [3,4,5]
num_filters = 3
dropout_keep_prob = 0.5
l2_reg_lambda = 0.0
l2_loss = 0.0

# Training
lr = 0.5
training=False
n_epochs = 3
skip_step = 1
global_step = tf.get_variable('global_step', initializer=tf.constant(0), trainable=False)

In [4]:
with tf.variable_scope('data'):
    # Read in train and dev data (sentences as word ids and padded, labels as one-hot vectors)
    #train_data, test_data = utils.get_vardial_dataset_from_many_files(batch_size, data_folder, data_files)
    #x_train, x_dev, y_train, y_dev = utils.get_vardial_dataset_from_many_files(data_folder, data_files)
    x_train, x_dev, y_train, y_dev = utils.get_vardial_dataset_from_train_dev_file(data_folder, train_file, dev_file)

    #Create datasets
    train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    train_data = train_data.shuffle(10000) # to shuffle your data
    train_data = train_data.batch(batch_size)

    test_data = tf.data.Dataset.from_tensor_slices((x_dev, y_dev))
    test_data = test_data.batch(batch_size)

    # Create iterators
    iterator = tf.data.Iterator.from_structure(train_data.output_types, 
                                           train_data.output_shapes)
    sentence, label = iterator.get_next()
    # shape = [batch_size, sentence_length],[batch_size, num_classes]

    sentence_length = sentence.shape[1].value
    num_classes = label.shape[1].value
    n_test = len(y_dev)

    train_init = iterator.make_initializer(train_data)  # initializer for train_data
    test_init = iterator.make_initializer(test_data)    # initializer for test_data


Loading data...
Loading data...
Padding sentences...
Building word vocabulary...
Converting to ids...
Train/Dev split: 10/10
train shape: (10, 48)
dev shape: (10, 48)
vocab_size 131
sentence max words 48


In [5]:
""" Step 2 + 3: define weights and embedding lookup
"""
with tf.variable_scope('char_embed'):
    embed_matrix = tf.get_variable('embed_matrix', 
                                        shape=[vocab_size, embed_size],
                                        initializer=tf.random_uniform_initializer())
    embedded_chars = tf.nn.embedding_lookup(embed_matrix, sentence, name='embedding')
    embedded_chars_expanded = tf.expand_dims(embedded_chars, -1)
    """sess = tf.Session()
    init = tf.global_variables_initializer()
    sess.run(init)
    sess.run(train_init)"""
    print(embed_matrix.eval)
    print(embedded_chars.eval)
    print(embedded_chars_expanded.eval)

<bound method Variable.eval of <tf.Variable 'char_embed/embed_matrix:0' shape=(50000, 128) dtype=float32_ref>>
<bound method Tensor.eval of <tf.Tensor 'char_embed/embedding:0' shape=(?, 48, 128) dtype=float32>>
<bound method Tensor.eval of <tf.Tensor 'char_embed/ExpandDims:0' shape=(?, 48, 128, 1) dtype=float32>>


In [6]:
# Create a convolution + maxpool layer for each filter size
pooled_outputs = []
for i, filter_size in enumerate(filter_sizes):
    with tf.variable_scope("conv-maxpool-%s" % filter_size):

        # Convolution layer
        conv = tf.layers.conv2d(inputs=embedded_chars_expanded,
                          filters=num_filters,
                          kernel_size=[filter_size, embed_size],
                          padding='SAME',
                          activation=tf.nn.relu,
                          name='conv')
        print(conv.eval)
        # shape = [batch_size, sequence_length, embed_size, num_filters]
        
        # Maxpooling over the outputs
        pooled = tf.layers.max_pooling2d(inputs=conv, 
                                        pool_size=(sentence.shape[1].value - filter_size + 1, 1), 
                                        strides=[filter_size, embed_size],
                                        name='pool')
        print(pooled.eval)
        pooled_outputs.append(pooled)
         # shape =  [batch_size, 1, 1, num_filters]

<bound method Tensor.eval of <tf.Tensor 'conv-maxpool-3/conv/Relu:0' shape=(?, 48, 128, 3) dtype=float32>>
<bound method Tensor.eval of <tf.Tensor 'conv-maxpool-3/pool/MaxPool:0' shape=(?, 1, 1, 3) dtype=float32>>
<bound method Tensor.eval of <tf.Tensor 'conv-maxpool-4/conv/Relu:0' shape=(?, 48, 128, 3) dtype=float32>>
<bound method Tensor.eval of <tf.Tensor 'conv-maxpool-4/pool/MaxPool:0' shape=(?, 1, 1, 3) dtype=float32>>
<bound method Tensor.eval of <tf.Tensor 'conv-maxpool-5/conv/Relu:0' shape=(?, 48, 128, 3) dtype=float32>>
<bound method Tensor.eval of <tf.Tensor 'conv-maxpool-5/pool/MaxPool:0' shape=(?, 1, 1, 3) dtype=float32>>


In [7]:
num_filters_total = num_filters * len(filter_sizes)
print(num_filters_total)
h_pool = tf.concat(pooled_outputs, 3)
h_pool_flat = tf.reshape(h_pool, [-1, num_filters_total])
print(h_pool.eval)
print(h_pool_flat.eval)

9
<bound method Tensor.eval of <tf.Tensor 'concat:0' shape=(?, 1, 1, 9) dtype=float32>>
<bound method Tensor.eval of <tf.Tensor 'Reshape:0' shape=(?, 9) dtype=float32>>


In [8]:
with tf.name_scope("dropout"):
   h_drop = tf.nn.dropout(h_pool_flat, dropout_keep_prob)
print(h_drop.eval)

<bound method Tensor.eval of <tf.Tensor 'dropout/dropout/mul:0' shape=(?, 9) dtype=float32>>


In [9]:
with tf.variable_scope("fully-connected"):
    scores = tf.layers.dense(h_drop, num_classes,
                kernel_initializer=tf.contrib.layers.xavier_initializer())
    predictions = tf.argmax(scores, 1, name="predictions")
print(scores.eval)
print(predictions.eval)

<bound method Tensor.eval of <tf.Tensor 'fully-connected/dense/BiasAdd:0' shape=(?, 5) dtype=float32>>
<bound method Tensor.eval of <tf.Tensor 'fully-connected/predictions:0' shape=(?,) dtype=int64>>


In [10]:
with tf.variable_scope('loss'):
    losses = tf.nn.softmax_cross_entropy_with_logits(logits=scores, labels=label)
    loss = tf.reduce_mean(losses) + l2_reg_lambda * 0.1
print(losses.eval)
print(loss.eval)

<bound method Tensor.eval of <tf.Tensor 'loss/Reshape_2:0' shape=(?,) dtype=float32>>
<bound method Tensor.eval of <tf.Tensor 'loss/add:0' shape=() dtype=float32>>


In [11]:
with tf.name_scope("accuracy"):
    correct_predictions = tf.equal(predictions, tf.argmax(label, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")
print(correct_predictions.eval)
print(accuracy.eval)

<bound method Tensor.eval of <tf.Tensor 'accuracy/Equal:0' shape=(?,) dtype=bool>>
<bound method Tensor.eval of <tf.Tensor 'accuracy/accuracy:0' shape=() dtype=float32>>


In [12]:
with tf.variable_scope('optimizer'):
    optimizer = tf.train.GradientDescentOptimizer(lr).minimize(loss, 
                                                      global_step=global_step)

In [13]:
with tf.variable_scope('summaries'):
    tf.summary.scalar('loss', loss)
    tf.summary.scalar('accuracy', accuracy)
    tf.summary.histogram('histogram_loss', loss)
    summary_op = tf.summary.merge_all()

In [14]:
def train_one_epoch(sess, saver, init, writer, epoch, step):
    start_time = time.time()
    sess.run(init) 
    training = True
    total_loss = 0
    n_batches = 0
    try:
        while True:
            _, l, acc, summaries = sess.run([optimizer, loss, accuracy, summary_op])
            writer.add_summary(summaries, global_step=step)
            if (step + 1) % skip_step == 0:
                print('step {0}, loss {1}'.format(step, l))
            step += 1
            total_loss += l
            n_batches += 1
    except tf.errors.OutOfRangeError:
        pass
    saver.save(sess, 'checkpoints/char-cnn', step)
    print('epoch {0}, loss {1}, took {2}'.format(epoch, total_loss/n_batches, time.time() - start_time))
    return step

def eval_once(sess, init, writer, epoch, step):
    start_time = time.time()
    sess.run(init)
    training = False
    total_correct_preds = 0
    try:
        while True:
            accuracy_batch, summaries = sess.run([accuracy, summary_op])
            writer.add_summary(summaries, global_step=step)
            total_correct_preds += accuracy_batch
    except tf.errors.OutOfRangeError:
        pass

    print('epoch {0}, accuracy {1}, took {2}'.format(epoch, total_correct_preds, time.time() - start_time))


In [15]:
utils.safe_mkdir('checkpoints')
utils.safe_mkdir('checkpoints/char-cnn')
writer = tf.summary.FileWriter('./graphs/char-cnn', tf.get_default_graph())

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    ckpt = tf.train.get_checkpoint_state(os.path.dirname('checkpoints/char-cnn/checkpoint'))
    if ckpt and ckpt.model_checkpoint_path:
        saver.restore(sess, ckpt.model_checkpoint_path)

    step = global_step.eval()

    for epoch in range(n_epochs):
        print('epoch {0}'.format(epoch))
                step = train_one_epoch(sess, saver, train_init, writer, epoch, step)
        eval_once(sess, test_init, writer, epoch, step)
writer.close()

epoch 0
step 0, loss 1.9731290340423584
epoch 0, loss 1.9731290340423584, took 0.46268558502197266
epoch 0, accuracy 0.20000000298023224, took 0.12839317321777344
epoch 1
step 1, loss 1.7951428890228271
epoch 1, loss 1.7951428890228271, took 0.4263486862182617
epoch 1, accuracy 0.4000000059604645, took 0.1185295581817627
epoch 2
step 2, loss 1.598291277885437
epoch 2, loss 1.598291277885437, took 0.42763376235961914
epoch 2, accuracy 0.6000000238418579, took 0.1194753646850586
